In [2]:
# 引入所需库
import numpy as np
import pandas as pd
import time
import copy

In [3]:
# 读取文件及数据初始化
pt_tmp=pd.read_excel("JSP_dataset.xlsx",sheet_name="Processing Time",index_col =[0])
ms_tmp=pd.read_excel("JSP_dataset.xlsx",sheet_name="Machines Sequence",index_col =[0])

dfshape=pt_tmp.shape
num_mc=dfshape[1] # 机器台数
num_job=dfshape[0] # 工序数
num_gene=num_mc*num_job # 单条染色体中的基因数

pt=[list(map(int, pt_tmp.iloc[i])) for i in range(num_job)]
ms=[list(map(int,ms_tmp.iloc[i])) for i in range(num_job)]


In [4]:
# 输入参数设置
"""
population_size=int(input('Please input the size of population: ') or 30) 
crossover_rate=float(input('Please input the size of Crossover Rate: ') or 0.8) 
mutation_rate=float(input('Please input the size of Mutation Rate: ') or 0.2) 
mutation_selection_rate=float(input('Please input the mutation selection rate: ') or 0.2)
num_mutation_jobs=round(num_gene*mutation_selection_rate)
num_iteration=int(input('Please input number of iteration: ') or 2000) 
    
start_time = time.time()
""" 

"\npopulation_size=int(input('Please input the size of population: ') or 30) \ncrossover_rate=float(input('Please input the size of Crossover Rate: ') or 0.8) \nmutation_rate=float(input('Please input the size of Mutation Rate: ') or 0.2) \nmutation_selection_rate=float(input('Please input the mutation selection rate: ') or 0.2)\nnum_mutation_jobs=round(num_gene*mutation_selection_rate)\nnum_iteration=int(input('Please input number of iteration: ') or 2000) \n    \nstart_time = time.time()\n"

In [5]:
# 设定初始参数
POPULATION_SIZE = 30 # 种群大小
CROSSOVER_RATE = 0.8 # 交叉概率
MUTATION_RATE = 0.2 # 变异概率
MUTATION_SELECTION_RATE = 0.2 # 变异选择率
NUM_MUTATION_JOBS = round(num_gene*MUTATION_SELECTION_RATE) # 变异基因数
NUM_ITERATION = 2000 # 迭代次数
ckeck_time = False # 是否计算运行时间

In [6]:
# 开始计时
if ckeck_time:
    start_time = time.time()

In [7]:
# 生成初始种群
Tbest=999999999999999 # 用于记录最优解
best_list,best_obj=[],[] # 用于记录最优解的染色体和目标函数值
population_list=[] # 用于存储种群
for i in range(POPULATION_SIZE):
    nxm_random_num=list(np.random.permutation(num_gene)) # 生成一个随机序列
    population_list.append(nxm_random_num) # 添加到种群中
    for j in range(num_gene):
        population_list[i][j]=population_list[i][j]%num_job # 生成随机基因序列

In [8]:
# 生成新一代
for n in range(NUM_ITERATION):
    Tbest_now=99999999999           
   
    # 交配
    parent_list=copy.deepcopy(population_list)      # 深拷贝父母染色体
    offspring_list=copy.deepcopy(population_list)   # 深拷贝子代染色体
    S=list(np.random.permutation(POPULATION_SIZE))  # 返回随机的序列，用于选择交配父母染色体
    
    for m in range(int(POPULATION_SIZE/2)):         # 交配次数
        crossover_prob=np.random.rand()             # 生成随机交叉概率
        if CROSSOVER_RATE>=crossover_prob:          # 交叉
            parent_1= population_list[S[2*m]][:]    # 相邻两个进行交叉
            parent_2= population_list[S[2*m+1]][:]
            child_1=parent_1[:]
            child_2=parent_2[:]
            cutpoint=list(np.random.choice(num_gene, 2, replace=False)) # 生成两个随机的交叉点
            cutpoint.sort() # 交叉点排序
        
            child_1[cutpoint[0]:cutpoint[1]]=parent_2[cutpoint[0]:cutpoint[1]]
            child_2[cutpoint[0]:cutpoint[1]]=parent_1[cutpoint[0]:cutpoint[1]]
            offspring_list[S[2*m]]=child_1[:]
            offspring_list[S[2*m+1]]=child_2[:]
    
    # 修复
    for m in range(POPULATION_SIZE):
        job_count={}
        larger,less=[],[] # 'larger' record jobs appear in the chromosome more than m times, and 'less' records less than m times.
        for i in range(num_job):
            if i in offspring_list[m]:
                count=offspring_list[m].count(i)
                pos=offspring_list[m].index(i)
                job_count[i]=[count,pos] # store the above two values to the job_count dictionary
            else:
                count=0
                job_count[i]=[count,0]
            if count>num_mc:
                larger.append(i)
            elif count<num_mc:
                less.append(i)
                
        for k in range(len(larger)):
            chg_job=larger[k]
            while job_count[chg_job][0]>num_mc:
                for d in range(len(less)):
                    if job_count[less[d]][0]<num_mc:                    
                        offspring_list[m][job_count[chg_job][1]]=less[d]
                        job_count[chg_job][1]=offspring_list[m].index(chg_job)
                        job_count[chg_job][0]=job_count[chg_job][0]-1
                        job_count[less[d]][0]=job_count[less[d]][0]+1                    
                    if job_count[chg_job][0]==num_mc:
                        break     